In [3]:
!pip install openai-whisper --no-cache-dir
!pip install pydub
!pip install psutil
!pip install ffmpeg

import os
import whisper
import gc
from pydub import AudioSegment
import tempfile
from tqdm.notebook import tqdm
import psutil
import time  # Precisamos importar o módulo 'time' para usar 'time.sleep()'

In [4]:
def split_audio(file_name, chunk_length_ms=300000):
    audio = AudioSegment.from_file(file_name, format="mp3")
    return [audio[i:i+chunk_length_ms] for i in range(0, len(audio), chunk_length_ms)]

def split_and_transcribe_audio(model, file_name, output_file, chunk_length_ms=300000):
    chunks = split_audio(file_name, chunk_length_ms)
    with open(output_file, 'w', encoding='utf-8') as f_out:
        for chunk_num, audio_chunk in enumerate(chunks):
            temp_file = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
            try:
                audio_chunk.export(temp_file.name, format="wav")
                retry_attempts = 3
                delay = 2  # delay inicial em segundos
                success = False
                while retry_attempts > 0:
                    try:
                        result = model.transcribe(temp_file.name, language="Portuguese")
                        f_out.write(result["text"])
                        success = True
                        break
                    except MemoryError:
                        print(f"Erro de memória no chunk {chunk_num}. Tentando novamente em {delay} segundos...")
                        retry_attempts -= 1
                        time.sleep(delay)
                        delay *= 2  # Double the delay for the next retry
                    except Exception as e:
                        print(f"Erro no chunk {chunk_num}: {e}")
                        break

                if not success:
                    print(f"Não foi possível transcrever o chunk {chunk_num} após várias tentativas.")

                if chunk_num % 20 == 0 and chunk_num > 0:
                    gc.collect()
                    mem = psutil.virtual_memory()
                    while mem.available < 1 * (1024 ** 3):  # espera até ter pelo menos 1GB disponível
                        print(f"Memória baixa ({mem.available / (1024.0 ** 3):.2f} GB). Esperando...")
                        time.sleep(10)
                        mem = psutil.virtual_memory()

            finally:
                temp_file.close()
                os.unlink(temp_file.name)

def main():
    model = whisper.load_model("large")
    files = [
        "./sample_data/audio-mic.mp3",]



    for file_path in files:
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        output_file = os.path.join("./sample_data", f"{base_name}.txt")
        split_and_transcribe_audio(model, file_path, output_file)
        print(f"Transcrição para o arquivo '{file_path}' salva em '{output_file}'.")

if __name__ == "__main__":
    main()


100%|██████████████████████████████████████| 2.87G/2.87G [00:18<00:00, 163MiB/s]


Transcrição para o arquivo './sample_data/audio-mic.mp3' salva em './sample_data/audio-mic.txt'.


In [ ]:
import os
import zipfile

def compactar_arquivos_transcricao(diretorio, nome_zip_saida):
    """
    Compacta todos os arquivos .txt em um diretório específico em um arquivo zip.

    Args:
    - diretorio (str): O diretório de onde os arquivos .txt serão coletados.
    - nome_zip_saida (str): O nome do arquivo zip de saída.
    """
    with zipfile.ZipFile(nome_zip_saida, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(diretorio):
            for file in files:
                if file.endswith(".txt"):
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, diretorio)
                    zipf.write(file_path, arcname)

def main():
    diretorio = "./sample_data/sections"  # Diretório onde estão os arquivos de transcrição
    nome_zip_saida = "transcricoes_audios.zip"  # Nome do arquivo zip de saída

    compactar_arquivos_transcricao(diretorio, nome_zip_saida)
    print(f"Arquivos de transcrição foram compactados em '{nome_zip_saida}'.")

if __name__ == "__main__":
    main()